In [1]:
!pip install optuna dagshub mlflow

  Using cached optuna-4.6.0-py3-none-any.whl.metadata (17 kB)
  Using cached dagshub-0.6.3-py3-none-any.whl.metadata (12 kB)
  Using cached alembic-1.17.2-py3-none-any.whl.metadata (7.2 kB)
  Using cached colorlog-6.10.1-py3-none-any.whl.metadata (11 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached dacite-1.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached gql-4.0.0-py3-none-any.whl.metadata (10 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached treelib-1.8.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pathvalidate-3.3.1-py3-none-any.whl.metadata (12 kB)
  Using cached semver-3.0.4-py3-none-any.whl.metadata (6.8 kB)
  Using cached dagshub_annotation_converter-0.1.15-py3-none-any.whl.metadata (3.2 kB)
  Using cached flask_cors-6.0.1-py3-none-any.whl.metadata (5.3 kB)
  Usi

In [2]:
!pip install category_encoders

  Using cached category_encoders-2.9.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached patsy-1.0.2-py2.py3-none-any.whl.metadata (3.6 kB)
Using cached category_encoders-2.9.0-py3-none-any.whl (85 kB)
Using cached patsy-1.0.2-py2.py3-none-any.whl (233 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.6 MB 7.6 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/9.6 MB 4.0 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/9.6 MB 4.0 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.6 MB 3.0 MB/s eta 0:00:03
   ------------ --------------------------- 2.9/9.6 MB 3.1 MB/s eta 0:00:03
   ----------------- ---------------------- 4.2/9.6 MB 3.3 MB/s eta 0:00:02
   -------------------- ------------------- 5.0/9.6 MB 3.4 MB/s eta 0:00:02
   ------------------------ --------------- 5.8/9.6 MB 3.5 MB/s eta 0:00:02
   -------------------------- ------------- 6.3/9.6 MB 3.5 MB/s et

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler, FunctionTransformer
import category_encoders as ce

In [2]:
import dagshub
dagshub.init(repo_owner='sourav664', repo_name='real-estate-hybrid-app', mlflow=True)


Accessing as sourav664

Initialized MLflow to track repo "sourav664/real-estate-hybrid-app"

Repository sourav664/real-estate-hybrid-app initialized!

In [3]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow")

In [4]:
# mlflow experiment

mlflow.set_experiment("Exp 5 - Neural Network Hyperparameter Tuning")

<Experiment: artifact_location='mlflow-artifacts:/2780ecf95a884743ad857732861adc93', creation_time=1764997677581, experiment_id='5', last_update_time=1764997677581, lifecycle_stage='active', name='Exp 5 - Neural Network Hyperparameter Tuning', tags={}>

In [5]:

from sklearn import set_config

set_config(transform_output="pandas")

In [6]:
from pathlib import Path

path = Path.cwd().parent
data_path = path / 'data/raw'

In [7]:
df = pd.read_csv(f"{data_path}/final_data.csv")

In [8]:
df.duplicated().sum(
    
)

np.int64(5808)

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
X = df.drop(columns=["price"])
y = df["price"]

In [11]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
     

In [12]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (33540, 11)
The shape of test data is (8386, 11)


In [13]:
ohe_encode = ['transactiontype','region','propertytype','furnished','ageofcons']
target_encode = ['locality']
robust_scaler = ['bedrooms','bathrooms','balconies','superbuiltupareasqft']

In [14]:
from sklearn.preprocessing import FunctionTransformer

pt = FunctionTransformer(np.log1p, inverse_func=np.expm1, validate=True)
y_train_pt = pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt = pt.transform(y_test.values.reshape(-1,1))


c:\Users\soura\anaconda3\envs\deep_learning\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))
c:\Users\soura\anaconda3\envs\deep_learning\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))
c:\Users\soura\anaconda3\envs\deep_learning\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))


In [15]:
preprocessor = ColumnTransformer(
    [
        ('ohe_encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ohe_encode),
        ('robust_scaler', RobustScaler(), robust_scaler)
    ],remainder="passthrough", n_jobs=-1, force_int_remainder_cols=False,verbose_feature_names_out=False
)


preprocessor.set_output(transform="pandas")

,transformers,"[('ohe_encode', ...), ('robust_scaler', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,False
,categories,'auto'
,drop,'first'
,sparse_output,False


In [16]:
# Creating a pipeline
pipeline = Pipeline([
    ('target_encoder', ce.TargetEncoder(cols=target_encode)),
    ('preprocessor', preprocessor)
    
])

pipeline.fit(X_train,y_train)

c:\Users\soura\anaconda3\envs\deep_learning\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


,steps,"[('target_encoder', ...), ('preprocessor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,verbose,0
,cols,['locality']
,drop_invariant,False
,return_df,True
,handle_missing,'value'
,handle_unknown,'value'
,min_samples_leaf,20


In [17]:
X_train_transformed = pipeline.transform(X_train)
X_test_transformed = pipeline.transform(X_test)

In [18]:
import torch
print(torch.__version__)
import torch.nn as nn
print(nn)


2.8.0+cu128
<module 'torch.nn' from 'c:\\Users\\soura\\anaconda3\\envs\\deep_learning\\Lib\\site-packages\\torch\\nn\\__init__.py'>


In [19]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [20]:
y_train_pt

array([[1.0919233 ],
       [0.43825493],
       [1.38128182],
       ...,
       [1.58923521],
       [1.178655  ],
       [2.30258509]], shape=(33540, 1))

In [21]:

from torch.utils.data import Dataset , DataLoader

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [22]:
train_dataset = CustomDataset(X_train_transformed,y_train_pt)
test_dataset = CustomDataset(X_test_transformed,y_test_pt)

In [37]:
class MyNN(nn.Module):

  def __init__(self, input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate):

    super().__init__()

    layers = []

    for i in range(num_hidden_layers):

      layers.append(nn.Linear(input_dim, neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_dim = neurons_per_layer

    layers.append(nn.Linear(neurons_per_layer, output_dim))

    self.model = nn.Sequential(*layers)

  def forward(self, x):

    return self.model(x)

In [24]:
import torch.optim as optim
loss_function = nn.MSELoss()

In [31]:
X_train_transformed.shape[1]

22

In [38]:
# objective function
def objective(trial):

  # next hyperparameter values from the search space
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5)
  neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
  epochs = trial.suggest_int("epochs", 10, 50, step=10)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
  batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
  optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  # model init
  input_dim = 22
  output_dim = 1

  model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)
  model.to(device)

  # optimizer selection
  criterion = nn.MSELoss()
  optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

  if optimizer_name == 'Adam':
    optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimizer_name == 'SGD':
    optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
    optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  # training loop

  for epoch in range(epochs):

    for batch_features, batch_values in train_loader:

      # move data to gpu
      batch_features, batch_values = batch_features.to(device), batch_values.to(device)

      # forward pass
      outputs = model(batch_features)

      # calculate loss
      loss = criterion(outputs, batch_values)

      # back pass
      optimizer.zero_grad()
      loss.backward()

      # update grads
      optimizer.step()


  # evaluation
  model.eval()
  # evaluation on test data
  total = 0
  test_loss = 0

  with torch.inference_mode():

    for batch_features, batch_values in test_loader:

      # move data to gpu
      batch_features, batch_values = batch_features.to(device), batch_values.to(device)

      outputs = model(batch_features)

      loss = criterion(outputs, batch_values)
      test_loss += loss.item()
      
      

  mse_loss = test_loss / len(test_loader) 

  return mse_loss

In [39]:
import optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=30,n_jobs=-1,show_progress_bar=True)

[I 2025-12-06 13:52:26,385] A new study created in memory with name: no-name-d7348235-4242-4d12-bfa6-035cd4b638be


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-12-06 13:54:25,058] Trial 1 finished with value: 0.09791802437869768 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 64, 'epochs': 10, 'learning_rate': 0.07259786705810548, 'dropout_rate': 0.4, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 3.18010351453521e-05}. Best is trial 1 with value: 0.09791802437869768.
[I 2025-12-06 13:56:04,323] Trial 8 finished with value: 0.1020842891638026 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 112, 'epochs': 40, 'learning_rate': 0.0003228348853576997, 'dropout_rate': 0.4, 'batch_size': 128, 'optimizer': 'RMSprop', 'weight_decay': 1.457163578375208e-05}. Best is trial 1 with value: 0.09791802437869768.
[I 2025-12-06 13:56:17,211] Trial 6 finished with value: 0.0562143295280947 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 56, 'epochs': 20, 'learning_rate': 3.809709327038309e-05, 'dropout_rate': 0.1, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 0.000147697487420606}. Best i

In [40]:
study.best_params

{'num_hidden_layers': 3,
 'neurons_per_layer': 96,
 'epochs': 40,
 'learning_rate': 0.004654130990759847,
 'dropout_rate': 0.2,
 'batch_size': 32,
 'optimizer': 'RMSprop',
 'weight_decay': 2.902930141361924e-05}

In [28]:
study.best_value

0.045771055815904314

In [43]:
model = MyNN(input_dim=22,output_dim=1,neurons_per_layer=96,dropout_rate=0.2, num_hidden_layers=3)
model.to(device)

MyNN(
  (model): Sequential(
    (0): Linear(in_features=22, out_features=96, bias=True)
    (1): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=96, out_features=96, bias=True)
    (5): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=96, out_features=96, bias=True)
    (9): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=96, out_features=1, bias=True)
  )
)

In [31]:
from torchmetrics.functional import r2_score

In [41]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_r2_score = 0, 0
    
    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metrics across all batches
        r2_s  = r2_score(y_pred, y)
        train_r2_score += r2_s.item()
    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_r2_score = train_r2_score / len(dataloader)
    return train_loss, train_r2_score

In [44]:
def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval() 
    
    # Setup test loss and test accuracy values
    test_loss, test_r2_score = 0, 0
    
    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)
    
            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred, y)
            test_loss += loss.item()
            
            # Calculate and accumulate accuracy
            r2_s  = r2_score(test_pred, y)
            test_r2_score += r2_s.item()
            
    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_r2_score = test_r2_score / len(dataloader)
    return test_loss, test_r2_score

In [45]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.MSELoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_r2_score": [],
        "test_loss": [],
        "test_r2_score": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_r2_score = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_r2_score = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_r2_score: {train_r2_score:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_r2_score: {test_r2_score:.4f}"
        )

        # 5. Update results dictionary
        # Ensure all data is moved to CPU and converted to float for storage
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["train_r2_score"].append(train_r2_score.item() if isinstance(train_r2_score, torch.Tensor) else train_r2_score)
        results["test_loss"].append(test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss)
        results["test_r2_score"].append(test_r2_score.item() if isinstance(test_r2_score, torch.Tensor) else test_r2_score)

    # 6. Return the filled results at the end of the epochs
    return results

In [46]:
# Set random seeds
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)


# Set number of epochs
NUM_EPOCHS = 40



loss_fn = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.004654130990759847, weight_decay=2.902930141361924e-05)



from timeit import default_timer as timer 
start_time = timer()

# Train model_0 
model_0_results = train(model=model, 
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1161 | train_r2_score: 0.6575 | test_loss: 0.0777 | test_r2_score: 0.7741
Epoch: 2 | train_loss: 0.0812 | train_r2_score: 0.7585 | test_loss: 0.0675 | test_r2_score: 0.7935
Epoch: 3 | train_loss: 0.0740 | train_r2_score: 0.7803 | test_loss: 0.1187 | test_r2_score: 0.6522
Epoch: 4 | train_loss: 0.0738 | train_r2_score: 0.7814 | test_loss: 0.1089 | test_r2_score: 0.6739
Epoch: 5 | train_loss: 0.0715 | train_r2_score: 0.7902 | test_loss: 0.0574 | test_r2_score: 0.8275
Epoch: 6 | train_loss: 0.0714 | train_r2_score: 0.7928 | test_loss: 0.0803 | test_r2_score: 0.7312
Epoch: 7 | train_loss: 0.0713 | train_r2_score: 0.7899 | test_loss: 0.0513 | test_r2_score: 0.8122
Epoch: 8 | train_loss: 0.0716 | train_r2_score: 0.7872 | test_loss: 0.0571 | test_r2_score: 0.8048
Epoch: 9 | train_loss: 0.0709 | train_r2_score: 0.7904 | test_loss: 0.0555 | test_r2_score: 0.8182
Epoch: 10 | train_loss: 0.0704 | train_r2_score: 0.7921 | test_loss: 0.0706 | test_r2_score: 0.7883
Epoch: 11